# Deploy

In [1]:
!pygmentize sklearnserver-iris.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1
    svcOrchSpec:
      env:
      - name: SELDON_LOG_LEVEL
        value: DEBUG


In [2]:
!kubectl apply -f sklearnserver-iris.yaml

seldondeployment.machinelearning.seldon.io/sklearn configured


In [3]:
!kubectl rollout status deploy/iris-default-4903e3c

deployment "iris-default-4903e3c" successfully rolled out


In [4]:
!kubectl get pods

NAME                                    READY   STATUS    RESTARTS   AGE
ambassador-69d88f894c-47vmk             1/1     Running   0          97m
ambassador-69d88f894c-dfx6v             1/1     Running   0          97m
ambassador-69d88f894c-pmwr8             1/1     Running   0          97m
iris-default-4903e3c-5989684fcd-k4ff9   2/2     Running   0          10m


# Test

In [5]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="sklearn",namespace="seldon")

### ProtoBuffer

In [6]:
r = sc.predict(gateway="ambassador",transport="rest", shape=(2, 4))
print(r)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 2
    shape: 4
    values: 0.4911234968962078
    values: 0.3568807851045721
    values: 0.4890809724631734
    values: 0.7613342733273581
    values: 0.8852950122237542
    values: 0.6117993348836631
    values: 0.23762388564314052
    values: 0.9107943919487682
  }
}

Response:
meta {
  puid: "froua0na6ap88kmkh1hsqt9tcj"
  requestPath {
    key: "classifier"
    value: "seldonio/sklearnserver_rest:0.2"
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  tensor {
    shape: 2
    shape: 3
    values: 0.20581367410452373
    values: 0.3439156619906966
    values: 0.4502706639047797
    values: 0.4371093066169174
    values: 0.2901662855689947
    values: 0.2727244078140879
  }
}



In [7]:
from seldon_core.utils import get_data_from_proto
get_data_from_proto(r.response)

array([[0.20581367, 0.34391566, 0.45027066],
       [0.43710931, 0.29016629, 0.27272441]])

### Dictionary

In [8]:
r = sc.predict(gateway="ambassador", transport="rest", shape=(2, 4), client_return_type='dict')
print(r)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 2
    shape: 4
    values: 0.6118486035311628
    values: 0.8077893489327292
    values: 0.7110402326073282
    values: 0.28700131396824335
    values: 0.14721484327205991
    values: 0.20979876056386182
    values: 0.21151170790961282
    values: 0.29005218262915133
  }
}

Response:
{'meta': {'puid': 'cjphe7omoqhunpiadtnl56g9k3', 'tags': {}, 'routing': {}, 'requestPath': {'classifier': 'seldonio/sklearnserver_rest:0.2'}, 'metrics': []}, 'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [2, 3], 'values': [0.3650666063239941, 0.41793390602977126, 0.21699948764623464, 0.31673044846221393, 0.42967093437250325, 0.25359861716528276]}}}


In [9]:
from pprint import pprint
pprint(r.response)

{'data': {'names': ['t:0', 't:1', 't:2'],
          'tensor': {'shape': [2, 3],
                     'values': [0.3650666063239941,
                                0.41793390602977126,
                                0.21699948764623464,
                                0.31673044846221393,
                                0.42967093437250325,
                                0.25359861716528276]}},
 'meta': {'metrics': [],
          'puid': 'cjphe7omoqhunpiadtnl56g9k3',
          'requestPath': {'classifier': 'seldonio/sklearnserver_rest:0.2'},
          'routing': {},
          'tags': {}}}
